In [1]:
import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [18]:
loader = PyPDFLoader("../data/book.pdf")
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100)
texts = splitter.split_documents(
    documents
)
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="db"
)
vectordb.persist()

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
from langchain_openai import ChatOpenAI
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI( model_name="gpt-3.5-turbo", temperature=0), 
    chain_type="stuff", 
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
)

In [25]:
queries = ["Чия типологія поділяється на традиційні, харизматичні й раціональні системи?", 
           "Які існують стабілізуючі фактори функціонування політичної системи?",
           "Які конституційні засади сучасної політичної системи України ви знаєте?"]
for query in queries:
    result = qa({"query": query})
    print(result["result"])
    print(result["source_documents"])
    print("-------------------")

/Users/sviatoslavbidzilia/anaconda3/envs/pdf-rag/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Ця типологія поділяється за Максом Вебером.
[Document(page_content='традиціями, системою розділення праці, характером комунікацій тощо. Цей підхід дає змогу зіставляти', metadata={'page': 266, 'source': '/Users/sviatoslavbidzilia/PycharmProjects/pdf-rag/data/book.pdf'}), Document(page_content='харизматичні й раціональні системи.', metadata={'page': 201, 'source': '/Users/sviatoslavbidzilia/PycharmProjects/pdf-rag/data/book.pdf'}), Document(page_content='перші два типи притаманні стабільним соціальним спільнотам, то харизматична', metadata={'page': 331, 'source': '/Users/sviatoslavbidzilia/PycharmProjects/pdf-rag/data/book.pdf'}), Document(page_content='до того, що одні й ті ж самі терміни прихильниками різних політичних поглядів сприймаються', metadata={'page': 503, 'source': '/Users/sviatoslavbidzilia/PycharmProjects/pdf-rag/data/book.pdf'})]
-------------------
Стабілізуючі фактори функціонування політичної системи можуть включати в себе наявність демократичних інститутів, правову де